In [ ]:
# Análisis no supervisado, espacio de trabajo
# Instalación de programación acelerada por GPU
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda110,transformers,lookups]
!python -m spacy download es_dep_news_trf
!pip install torch===1.7.1+cu110 torchvision===0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html --user
!pip install cupy-cuda110
!pip install thinc-gpu-ops
!pip install -q wordcloud
!pip install pyLDAvis

import pandas as pd
import numpy as np
import spacy
from spacy.matcher import Matcher
import wordcloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from itertools import chain
from gensim import Phrases, corpora, models
import matplotlib
import pyLDAvis
import pyLDAvis.gensim
import sys
#Antes de hacer esto necesitamos importar el archivo de funciones que se encuentra en el repositorio.
sys.path.append('../')
import funciones as fn

In [ ]:
path='/content/drive/MyDrive/Colab Notebooks/Copia de estudiante_2.xlsx'
df=pd.read_excel(path)

df.head(8)

,ID,Snippet,Victima,Victimario,Xenofobia,Denuncia,Positiva
0,16547,..crítico. El agresor fue capturado e identifi...,NaN,1.0,NaN,NaN,NaN
1,16548,@EnriquePenalosa Esa sociedad es civilizada. A...,NaN,1.0,1.0,NaN,NaN
2,16549,"Cuánto pseudoprogresista xenófobo y clasista,c...",NaN,NaN,NaN,1.0,1.0
3,16550,"Perdón amigos venecos, pero el diablito no me ...",NaN,NaN,1.0,NaN,NaN
4,16551,El texto del reconocimiento señala: “Víctor Ba...,NaN,NaN,NaN,NaN,1.0
5,16552,@WSJForero Y además si el Gobierno le da esa v...,NaN,1.0,1.0,NaN,NaN
6,16553,"asustados, que volvimos a escuchar los sonidos...",NaN,NaN,NaN,NaN,NaN
7,16554,#Colombia | El presidente @IvanDuque presentó ...,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Eliminamos los URL: Todo lo que empieza por  HTTP
df['Clean_Text']=df['Snippet'].replace(r"http\S+", '', regex=True)
#Eliminamos todos los hashtags
df['Clean_Text']=df['Clean_Text'].replace('#\\w+', '', regex=True)
#Eliminamos los signos de puntuación repetidos dentro de los tweets
df['Clean_Text']=df['Clean_Text'].replace(r'([^0-9a-zA-Z])\1+', r'\1', regex=True)
#Eliminamos los signos de puntuación al inicio de un tweet
df['Clean_Text']=df['Clean_Text'].replace(r'^[^0-9a-zA-Z@#]+', '', regex=True)
#Eliminamos los dobles espacios
df['Clean_Text']=df['Clean_Text'].replace(r'()\1+', r'\1\1', regex=True)
df=df.reset_index()
df.head()

,index,ID,Snippet,Victima,Victimario,Xenofobia,Denuncia,Positiva,Clean_Text
0,0,16547,..crítico. El agresor fue capturado e identifi...,NaN,1.0,NaN,NaN,NaN,crítico. El agresor fue capturado e identifica...
1,1,16548,@EnriquePenalosa Esa sociedad es civilizada. A...,NaN,1.0,1.0,NaN,NaN,@EnriquePenalosa Esa sociedad es civilizada. A...
2,2,16549,"Cuánto pseudoprogresista xenófobo y clasista,c...",NaN,NaN,NaN,1.0,1.0,"Cuánto pseudoprogresista xenófobo y clasista,c..."
3,3,16550,"Perdón amigos venecos, pero el diablito no me ...",NaN,NaN,1.0,NaN,NaN,"Perdón amigos venecos, pero el diablito no me ..."
4,4,16551,El texto del reconocimiento señala: “Víctor Ba...,NaN,NaN,NaN,NaN,1.0,El texto del reconocimiento señala: “Víctor Ba...


In [ ]:
# Le decimos a spaCy que vamos con toda
spacy.prefer_gpu
# Corremos los datos en el core de español
nlp = spacy.load("es_core_news_sm", exclude=["ner"])
#Vemos cuales son los componentes del pipeline
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer']


In [ ]:
%time
#Aplicamos el pipe a cada objeto de nuestra lista de tweets
docs = list(nlp.pipe(df.Clean_Text.to_list()))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [ ]:
#TOKENIZATION
tokens=[]
for doc in docs:
    tokens.append([(token.text) for token in doc])

tokens=np.array(tokens)
tokens=pd.DataFrame(tokens).reset_index()
tokens.columns=['index', 'tokens']
tokens.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


,index,tokens
0,0,"[crítico, ., El, agresor, fue, capturado, e, i..."
1,1,"[@EnriquePenalosa, Esa, sociedad, es, civiliza..."
2,2,"[Cuánto, pseudoprogresista, xenófobo, y, clasi..."
3,3,"[Perdón, amigos, venecos, ,, pero, el, diablit..."
4,4,"[El, texto, del, reconocimiento, señala, :, “,..."


In [ ]:
#POS
pos=[]
for doc in docs:
    pos.append([(token.text, token.pos_) for token in doc])

pos=np.array(pos)
pos=pd.DataFrame(pos).reset_index()
pos.columns=['index', 'pos']
pos.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


,index,pos
0,0,"[(crítico, ADJ), (., PUNCT), (El, DET), (agres..."
1,1,"[(@EnriquePenalosa, ADJ), (Esa, DET), (socieda..."
2,2,"[(Cuánto, PROPN), (pseudoprogresista, VERB), (..."
3,3,"[(Perdón, PROPN), (amigos, NOUN), (venecos, AD..."
4,4,"[(El, DET), (texto, NOUN), (del, ADP), (recono..."


In [ ]:
#LEMATIZER
lemma=[]
for doc in docs:
    lemma.append([(token.lemma_) for token in doc])

lemma=np.array(lemma)
lemma=pd.DataFrame(lemma).reset_index()
lemma.columns=['index', 'lemmatized']
lemma.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


,index,lemmatized
0,0,"[crítico, ., el, agresor, ser, capturar, e, id..."
1,1,"[@enriquepenalós, ese, sociedad, ser, civiliza..."
2,2,"[Cuánto, pseudoprogresistar, xenófobo, y, clas..."
3,3,"[Perdón, amigo, veneco, ,, pero, el, diablito,..."
4,4,"[el, texto, del, reconocimiento, señalar, :, “..."


In [ ]:
#MERGE LISTS INTO DF
df=pd.merge(df, tokens, how='inner', on='index')
df=pd.merge(df, pos, how='inner', on='index')
df=pd.merge(df, lemma, how='inner', on='index')
df=df.drop(['index'], axis=1)
df.head()

,index,ID,Snippet,Victima,Victimario,Xenofobia,Denuncia,Positiva,Clean_Text,tokens,pos,lemmatized
0,0,16547,..crítico. El agresor fue capturado e identifi...,NaN,1.0,NaN,NaN,NaN,crítico. El agresor fue capturado e identifica...,"[crítico, ., El, agresor, fue, capturado, e, i...","[(crítico, ADJ), (., PUNCT), (El, DET), (agres...","[crítico, ., el, agresor, ser, capturar, e, id..."
1,1,16548,@EnriquePenalosa Esa sociedad es civilizada. A...,NaN,1.0,1.0,NaN,NaN,@EnriquePenalosa Esa sociedad es civilizada. A...,"[@EnriquePenalosa, Esa, sociedad, es, civiliza...","[(@EnriquePenalosa, ADJ), (Esa, DET), (socieda...","[@enriquepenalós, ese, sociedad, ser, civiliza..."
2,2,16549,"Cuánto pseudoprogresista xenófobo y clasista,c...",NaN,NaN,NaN,1.0,1.0,"Cuánto pseudoprogresista xenófobo y clasista,c...","[Cuánto, pseudoprogresista, xenófobo, y, clasi...","[(Cuánto, PROPN), (pseudoprogresista, VERB), (...","[Cuánto, pseudoprogresistar, xenófobo, y, clas..."
3,3,16550,"Perdón amigos venecos, pero el diablito no me ...",NaN,NaN,1.0,NaN,NaN,"Perdón amigos venecos, pero el diablito no me ...","[Perdón, amigos, venecos, ,, pero, el, diablit...","[(Perdón, PROPN), (amigos, NOUN), (venecos, AD...","[Perdón, amigo, veneco, ,, pero, el, diablito,..."
4,4,16551,El texto del reconocimiento señala: “Víctor Ba...,NaN,NaN,NaN,NaN,1.0,El texto del reconocimiento señala: “Víctor Ba...,"[El, texto, del, reconocimiento, señala, :, “,...","[(El, DET), (texto, NOUN), (del, ADP), (recono...","[el, texto, del, reconocimiento, señalar, :, “..."


In [ ]:
#REMOVE STOPWORDS
spanish_stopwords=stopwords.words('spanish')+['ser']
df['words']=df['lemmatized'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() and token.lower() not in spanish_stopwords and len(token)>1])
df.head()

,index,ID,Snippet,Victima,Victimario,Xenofobia,Denuncia,Positiva,Clean_Text,tokens,pos,lemmatized,words
0,0,16547,..crítico. El agresor fue capturado e identifi...,NaN,1.0,NaN,NaN,NaN,crítico. El agresor fue capturado e identifica...,"[crítico, ., El, agresor, fue, capturado, e, i...","[(crítico, ADJ), (., PUNCT), (El, DET), (agres...","[crítico, ., el, agresor, ser, capturar, e, id...","[crítico, agresor, capturar, identificado, fél..."
1,1,16548,@EnriquePenalosa Esa sociedad es civilizada. A...,NaN,1.0,1.0,NaN,NaN,@EnriquePenalosa Esa sociedad es civilizada. A...,"[@EnriquePenalosa, Esa, sociedad, es, civiliza...","[(@EnriquePenalosa, ADJ), (Esa, DET), (socieda...","[@enriquepenalós, ese, sociedad, ser, civiliza...","[sociedad, civilizar, aquí, haber, poder, metr..."
2,2,16549,"Cuánto pseudoprogresista xenófobo y clasista,c...",NaN,NaN,NaN,1.0,1.0,"Cuánto pseudoprogresista xenófobo y clasista,c...","[Cuánto, pseudoprogresista, xenófobo, y, clasi...","[(Cuánto, PROPN), (pseudoprogresista, VERB), (...","[Cuánto, pseudoprogresistar, xenófobo, y, clas...","[cuánto, pseudoprogresistar, xenófobo, clasist..."
3,3,16550,"Perdón amigos venecos, pero el diablito no me ...",NaN,NaN,1.0,NaN,NaN,"Perdón amigos venecos, pero el diablito no me ...","[Perdón, amigos, venecos, ,, pero, el, diablit...","[(Perdón, PROPN), (amigos, NOUN), (venecos, AD...","[Perdón, amigo, veneco, ,, pero, el, diablito,...","[perdón, amigo, veneco, diablito, gustar, xdxd]"
4,4,16551,El texto del reconocimiento señala: “Víctor Ba...,NaN,NaN,NaN,NaN,1.0,El texto del reconocimiento señala: “Víctor Ba...,"[El, texto, del, reconocimiento, señala, :, “,...","[(El, DET), (texto, NOUN), (del, ADP), (recono...","[el, texto, del, reconocimiento, señalar, :, “...","[texto, reconocimiento, señalar, víctor, bauti..."


In [ ]:
#LDA
tokens = df['words'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

In [ ]:
np.random.seed(123456)
num_topics = 5
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, update_every=0, \
                                  alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

In [ ]:
for i, topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.026*"haber" + 0.008*"hacer" + 0.008*"año" + 0.008*"venezolana" + 0.007*"tener" + 0.007*"ir" + 0.007*"colombia" + 0.006*"deber" + 0.006*"si" + 0.006*"decir" + 0.005*"poder" + 0.005*"migrant" + 0.004*"venezuela" + 0.004*"hoy" + 0.004*"nacionalidad" + 0.004*"mujer" + 0.004*"país" + 0.003*"colombiano" + 0.003*"saber" + 0.003*"así"

1: 0.018*"colombia" + 0.015*"hacer" + 0.013*"país" + 0.012*"haber" + 0.011*"migrant" + 0.009*"tener" + 0.008*"venezuela" + 0.007*"venezolanos" + 0.007*"veneco" + 0.006*"colombiano" + 0.006*"llegar" + 0.006*"si" + 0.005*"mismo" + 0.005*"poder" + 0.005*"persona" + 0.004*"ir" + 0.004*"venezolana" + 0.004*"decir" + 0.003*"dar" + 0.003*"seguir"

2: 0.019*"haber" + 0.019*"tener" + 0.010*"país" + 0.008*"colombia" + 0.008*"ver" + 0.008*"poder" + 0.006*"si" + 0.005*"año" + 0.004*"persona" + 0.004*"hacer" + 0.004*"venezolana" + 0.004*"venezuela" + 0.004*"pueblo" + 0.004*"mujer" + 0.003*"llegar" + 0.003*"solo" + 0.003*"joven" + 0.003*"millón" + 0.003*"ciudad" + 0.003*

In [ ]:
lda_model[corpus[0]]

[(3, 0.99503106)]

In [ ]:
print(df.Snippet.loc[0])

..crítico. El agresor fue capturado e identificado como Félix Manuel Rivero Goitia de nacionalidad venezolana, el cual fue capturado y dejado a disposición de las autoridades competentes.


In [ ]:
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
marcados=df.loc[pd.isnull(df[['Victima', 'Victimario', 'Xenofobia', 'Denuncia', 'Positiva']]).all(axis=1)==False]
marcados.head()

,index,ID,Snippet,Victima,Victimario,Xenofobia,Denuncia,Positiva,Clean_Text,tokens,pos,lemmatized,words
0,0,16547,..crítico. El agresor fue capturado e identifi...,NaN,1.0,NaN,NaN,NaN,crítico. El agresor fue capturado e identifica...,"[crítico, ., El, agresor, fue, capturado, e, i...","[(crítico, ADJ), (., PUNCT), (El, DET), (agres...","[crítico, ., el, agresor, ser, capturar, e, id...","[crítico, agresor, capturar, identificado, fél..."
1,1,16548,@EnriquePenalosa Esa sociedad es civilizada. A...,NaN,1.0,1.0,NaN,NaN,@EnriquePenalosa Esa sociedad es civilizada. A...,"[@EnriquePenalosa, Esa, sociedad, es, civiliza...","[(@EnriquePenalosa, ADJ), (Esa, DET), (socieda...","[@enriquepenalós, ese, sociedad, ser, civiliza...","[sociedad, civilizar, aquí, haber, poder, metr..."
2,2,16549,"Cuánto pseudoprogresista xenófobo y clasista,c...",NaN,NaN,NaN,1.0,1.0,"Cuánto pseudoprogresista xenófobo y clasista,c...","[Cuánto, pseudoprogresista, xenófobo, y, clasi...","[(Cuánto, PROPN), (pseudoprogresista, VERB), (...","[Cuánto, pseudoprogresistar, xenófobo, y, clas...","[cuánto, pseudoprogresistar, xenófobo, clasist..."
3,3,16550,"Perdón amigos venecos, pero el diablito no me ...",NaN,NaN,1.0,NaN,NaN,"Perdón amigos venecos, pero el diablito no me ...","[Perdón, amigos, venecos, ,, pero, el, diablit...","[(Perdón, PROPN), (amigos, NOUN), (venecos, AD...","[Perdón, amigo, veneco, ,, pero, el, diablito,...","[perdón, amigo, veneco, diablito, gustar, xdxd]"
4,4,16551,El texto del reconocimiento señala: “Víctor Ba...,NaN,NaN,NaN,NaN,1.0,El texto del reconocimiento señala: “Víctor Ba...,"[El, texto, del, reconocimiento, señala, :, “,...","[(El, DET), (texto, NOUN), (del, ADP), (recono...","[el, texto, del, reconocimiento, señalar, :, “...","[texto, reconocimiento, señalar, víctor, bauti..."


In [ ]:
#LDA
m_tokens = marcados['words'].tolist()
m_bigram_model = Phrases(m_tokens)
m_trigram_model = Phrases(m_bigram_model[m_tokens], min_count=1)
m_tokens = list(m_trigram_model[m_bigram_model[m_tokens]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
m_dictionary_LDA = corpora.Dictionary(m_tokens)
m_dictionary_LDA.filter_extremes(no_below=3)
m_corpus = [m_dictionary_LDA.doc2bow(tok) for tok in m_tokens]

In [ ]:
np.random.seed(123456)
num_topics = 5
%time m_lda_model = models.LdaModel(m_corpus, num_topics=num_topics, \
                                  id2word=m_dictionary_LDA, \
                                  passes=4, \
                                  alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(m_dictionary_LDA.keys()))

In [ ]:
for i, topic in m_lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.019*"haber" + 0.018*"colombia" + 0.017*"hacer" + 0.013*"veneco" + 0.012*"si" + 0.011*"poder" + 0.010*"niño" + 0.008*"país" + 0.008*"delincuente" + 0.008*"colombiano" + 0.007*"año" + 0.007*"trabajar" + 0.007*"venir" + 0.006*"deber" + 0.006*"persona" + 0.006*"ciudad" + 0.006*"asesinar" + 0.006*"ahora" + 0.006*"navidad_integración_conocer_abc" + 0.006*"buscar_mejor_oportunidad_cumplir"

1: 0.024*"haber" + 0.019*"tener" + 0.018*"colombia" + 0.017*"hacer" + 0.012*"país" + 0.011*"decir" + 0.010*"mujer" + 0.010*"pasar" + 0.009*"aquí" + 0.009*"si" + 0.009*"ver" + 0.008*"bogotá" + 0.008*"venezuela" + 0.007*"veneco" + 0.007*"seguir" + 0.006*"migrant_venezolano" + 0.006*"venezolanos" + 0.006*"gracias" + 0.006*"querer" + 0.006*"solo"

2: 0.016*"colombiano" + 0.015*"tener" + 0.008*"hoy" + 0.008*"poder" + 0.008*"niño" + 0.008*"hacer" + 0.008*"migrant" + 0.007*"aquí" + 0.007*"querer" + 0.007*"dar" + 0.006*"día" + 0.006*"gracias" + 0.006*"migrante" + 0.006*"venezuela" + 0.006*"decir" + 0.006*"mig

In [ ]:
for i in range(5): print(lda_model[corpus[i]], df.Snippet.loc[i])

[(3, 0.99503106)] ..crítico. El agresor fue capturado e identificado como Félix Manuel Rivero Goitia de nacionalidad venezolana, el cual fue capturado y dejado a disposición de las autoridades competentes.
[(1, 0.27978796), (3, 0.7182187)] @EnriquePenalosa Esa sociedad es civilizada. Aquí no hemos podido que en pocos metros de antejardín los vecinos recojan la caquita del perro. Que no dejen basura (esos son venezolanos o taxis) o se trepen a un árbol de durazno a saquearlo. Se volvería de okupas con más derechos que el dueño.
[(4, 0.99750775)] Cuánto pseudoprogresista xenófobo y clasista,como los uribistas:están en contra de la recepción de venezolanos pobres, les niegan derechos y después los culpan de todo.Por cálculo político,nadie los defiende porque el fascismo está en gente de toda corriente política.
[(4, 0.9804878)] Perdón amigos venecos, pero el diablito no me gusta xdxd
[(2, 0.99750775)] El texto del reconocimiento señala: “Víctor Bautista Olarte, quien con su vocación de se

In [ ]:
m_vis = pyLDAvis.gensim.prepare(topic_model=m_lda_model, corpus=m_corpus, dictionary=m_dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(m_vis)

In [ ]:
df.loc[pd.isnull(df[['Victima', 'Victimario', 'Xenofobia', 'Denuncia', 'Positiva']]).all(axis=1)==True]['Snippet'][11905]

'@AlejaUribeT_ Por los VENECOS , seres inservibles facilitas.'

In [ ]:
tokens = df.loc[pd.isnull(df[['Victima', 'Victimario', 'Xenofobia', 'Denuncia', 'Positiva']]).all(axis=1)==True]['tokens'][11905]
topics = m_lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in m_lda_model[m_dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])

,topic #,weight,words in topic
0,0,0.2,"0.019*""haber"" + 0.018*""colombia"" + 0.017*""hace..."
1,1,0.2,"0.024*""haber"" + 0.019*""tener"" + 0.018*""colombi..."
2,2,0.2,"0.016*""colombiano"" + 0.015*""tener"" + 0.008*""ho..."
3,3,0.2,"0.026*""veneco"" + 0.014*""haber"" + 0.011*""país"" ..."
4,4,0.2,"0.041*""haber"" + 0.014*""decir"" + 0.014*""país"" +..."


In [ ]:
df.loc[pd.isnull(df[['Victima', 'Victimario', 'Xenofobia', 'Denuncia', 'Positiva']]).all(axis=1)==True]['tokens'][10350:10400]

11874    [Cita, :, Iniciado, por, rolohotbta, Buenas, n...
11875    [@LauraMedinaRuiz, Los, venezolanos, venden, e...
11876    [He, perdido, absolutamente, la, confianza, en...
11877    [Witness, at, Arizona, Hearing, Offers, Bombsh...
11878    [PK21, Reacción, El, canal, de, difusión, para...
11879    [@MiguelRamirezcx, @IvanDuque, Si, fuese, de, ...
11880    [La, venezolana, Oriana, Altuve, se, lleva, un...
11881    [Capturan, a, dos, venezolanos, por, delito, d...
11882    [Gracias, al, Proyecto, Migración, Venezuela, ...
11883    [Cuál, es, el, cereal, venezolano, que, más, e...
11884    [Para, entender, el, alcance, del, Barómetro, ...
11885    [Venezolanos, estarían, siendo, amenazados, y,...
11886    [@gabocifuentes, @IvanDuque, Los, Venezolanos,...
11887         [JAJAJAJAJAJAJA, Popular, entre, venecos, .]
11888    [No, me, importa, qué, tan, atractivo, sea, ,,...
11889    [@jaimeflozada, @Legarda77443474, De, hecho, d...
11890    [Le, recomendamos, observar, a, sus, caballos,.